# Notebook for the development of module "scheduling_problem" of AI for carbon reduction project

# General model
We list below the common sets, parameters, variables and constraints that will be used in all the models.

## Sets
$$
T = \{0, ..., T_H-1\} = \text{Discrete time steps in the scheduling horizon (e.g., minutes).}
$$
$$
I = \{1, ...., N_M\} = \text{Set of machines.}
$$
$$
J_i = \{1, ..., N_i\} = \text{Set of jobs to be processed by machine $i$.}
$$

Other sets that will be used in the constraints:

$SHARED = \{M'$ : if $k,l \in M'$ then $k$ and $l$ have shared resources$\}$

$DEPEN = \{(k,l)$ : $l$ must start its job only after $k$ has finished its$\}$


## Parameters
For each machine:
$$
 e_{i} = \text{energy used by the machine $i$ per unit of time}
 $$
 $$
 f_{i} = \text{fixed cost for starting the machine $i$ at any time}
 $$
 $$
d_i = \text{duration of any of the job of machine $i$}
$$
$$
N_i = \text{number of jobs to complete for machine $i$}
$$
$$
c_i = \text{time of cooldown for machine $i$}
$$
$$
T_{j,i} = \text{threshold for job $j$ of machine $i$}
$$
For each time step:
$$
p_t = \text{energy produced at time $t$ by one solar panel}
$$
$$
m_t = \text{maximum energy that we can use at time $t$}
$$
For each solar panel:
$$
c_p = \text{cost of a solar panel}
$$
For each battery:
$$
c_b = \text{cost of a battery}
$$
$$
B = \text{capacity of a battery}
$$

## Variables
$$
x_{itj} = \text{if machine $i$ is working at time $t$ for job $j$}
$$
$$
y_{itj} = \text{if machine $i$ starts working at time $t$ for job $j$}
$$

## Constraints

1. Every machine $i \in I$ must do the required number $N_i$ of jobs with a certain duration $d_i$. For every $i \in I$ and $j \in J_i$:
$$
\sum_{t \in T} x_{itj} \ge  d_i
$$
2. Knapsack contraint that says that not all machines can run at the same time due to max power load for slot (parameter $m_t$, which could depend on time). For every $t \in T$:
$$
   \sum_{i \in I} \sum_{j \in J_i} e_{i}x_{itj} \le m_t
$$
3. Some machines may be allowed to run only at specific times for noise or maintenance windows / downtime or worker availability. For some $t \in T$ and $i \in I$:
   $$
   x_{itj} = 0 \, \, \forall j \in J_i
   $$
4. Shared resources: some machines can't run in the same time slot $t \in T$ because of shared resources. Given a set $SHARED = \{M'$ : if $k,l \in M'$ then $k$ and $l$ have shared resources$\}$ and $\forall t \in T$:
   $$
   \sum_{i \in M'} \sum_{j \in J_i} x_{itj} \le 1  \,\, \forall M' \in SHARED
   $$
5. A machine works only if it has just started or it was already working. For every $i \in I$, for every $j \in J_i$ and for every $t \in T$:
$$            
y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j}
$$
6. Machine dependencies: one machine starts only after another has finished. Given a set $DEPEN = \{(k,l)$ : $l$ must start its job only after $k$ has finished its$\}$ and for every time $t \in T$.
$$
y_{ltj} \cdot d_k \le  \sum_{t' < t, \, t' \in T} (x_{kt'j}) \text{ for every dependency }(k,l) \in DEPEN \text{ and for every $j \in J_l$}
$$
7. Cooldown Periods: some machines require a cooldown phase between uses (if this is not needed, set $c_i = 0$). For every $i \in I$ and for every $t \in T$:
$$
   c_i \cdot \sum_{j \in J_i} y_{itj} \le \sum_{t-c_i \le t' < t, \, t' \in T}\sum_{j \in J_i} (1-x_{it'j})
$$
8. Job deadlines: all jobs must finish before a specific time. For every $i \in I$ and for every $j \in J_i$, calling $T_{j,i}$ the threshold for job $j$ on machine $i$:
$$
     \sum_{t' \le T_{j,i}, \, t' \in T} x_{it'j} \ge d_i
$$
9. Only one job at a time. For every $i \in I$ and for every $t \in T$:
$$
    \sum_{j \in J_i} x_{itj} \le 1
$$
$$
    \sum_{j \in J_i} y_{itj} \le 1
$$


## PHASE 1
Determine the minimum number of solar panels ($M$) and battery units ($N$) for 100% self-sufficiency.

### Additional variables
$$
N = \text{how many batteries to buy}
$$
$$
M = \text{how many panels to buy}
$$
$$
s_t = \text{how much energy we accumulated till time $t$ in the batteries}
$$
### Objective function
$$
\min Nc_b+Mc_p
$$
### Additional constraints
1. No imported energy. For every $t \in T$:
$$
\sum_{i \in I}\sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le  0
$$
2. Constraints on the level of energy accumulated in the batteries. For every $t \in T$:
$$
         s_t = s_{t-1} + Mp_{t-1} - \sum_{i \in I}\sum_{j \in J_i} ( e_{i}x_{i(t-1)j} + f_i y_{i(t-1)j})
$$
$$
         0 \le s_t \le  NB
$$
$$
        s_0 = 0
$$

### Final model
The following is the final model that is solved in phase 1.

$$
\left\{
  \begin{array}{rcr}
        \min Nc_b+Mc_p \\
        \sum_{i \in I}\sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj}) - Mp_t - s_t \le & 0 & \forall t \in T \\
        s_t = & s_{t-1} + Mp_{t-1} - \sum_{i \in I}\sum_{j \in J_i} ( e_{i}x_{i(t-1)j} + f_i y_{i(t-1)j}) & \forall t \in T \\
        0   \le s_t \le & NB & \forall t \in T \\
        s_0 = & 0 \\
        \sum_{t \in T} x_{itj} \ge & d_i & \forall i \in I, \, \forall j \in J \\
        \sum_{i \in I} \sum_{j \in J_i} e_{i}x_{itj}  \le & m_t & \forall t \in T \\
        x_{itj} = & 0 & \text{for some $t \in T$ and for some $i \in I$, } \forall j \in J_i \\
        \sum_{i \in M'} \sum_{j \in J_i} x_{itj} \le & 1  & \forall t \in T, \, \forall M' \in SHARED\\
        y_{itj} \le x_{itj} \le & y_{itj}+x_{i(t-1)j} & \forall t \in T-\{0\}, \, \forall i \in I, \, \forall j \in J_i \\
        y_{ltj} \cdot d_k  \le & \sum_{t' < t, \, t' \in T} (x_{kt'j}) & \text{ for every dependency }(k,l) \in DEPEN \text{ and $\forall j \in J_l$, $\forall t \in T$} \\
        c_i \cdot \sum_{j \in J_i} y_{itj} \le & \sum_{t-c_i \le t' < t, \, t' \in T}\sum_{j \in J_i} (1-x_{it'j}) & \forall t \in T, \, \forall i \in I \\
        \sum_{t' \le T_{j,i}, \, t' \in T} x_{it'j} \ge & d_i & \forall i \in I, \, \forall j \in J_i\\
        \sum_{j \in J_i} x_{itj} \le & 1 & \forall t \in T, \, \forall i \in I\\
        \sum_{j \in J_i} y_{itj} \le & 1 & \forall t \in T, \, \forall i \in I \\
        x_{ijt} \in & \{0,1\} & \forall t \in T, \, \forall i \in I, \, \forall j \in J \\
        y_{ijt} \in & \{0,1\} & \forall t \in T, \, \forall i \in I, \, \forall j \in J \\
        s_t \in & \mathbb{R} & \forall t \in T \\
        N, \, M \in & \mathbb{N} 
  \end{array}
\right.
$$

## PHASE 2
Taking the number of panels $M^*$ and the number of batteries $N^*$ that result from phase 1, compute the number of years that our company could work continuing to pay external energy instead of buying solar panels.

Let $c_{annual}$ be the annual cost of the imported energy.

$$
horizon = H = \frac{c_bN^*+c_pM^*}{c_{annual}}
$$

## PHASE 3
Within the budget implied by Phase 2’s horizon, find optimal $N$ and $M$ to minimize total energy cost (capital + operational).

### Additional parameters
$$
T_H = H_{time \, units} = \text{temporal horizon from phase 2 in time units} 
$$
### Additional variables
$$
N = \text{how many batteries to buy}
$$
$$
M = \text{how many panels to buy}
$$
$$
s_t = \text{how much energy we accumulated till time $t$ in the batteries}
$$
$$
z_t = \text{imported energy at time t}
$$

### Objective function
$$
\min Nc_b + Mc_p + \sum_{t \in T} z_t
$$
### Additional constraints
1. Modified energy balance constraint. For every $t \in T$:
$$
\sum_{i \in I} \sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj}) - Mp_t - s_t \le z_t
$$
2. Constraints on the level of energy accumulated in the batteries. For every $t \in T$:
$$
         s_t = s_{t-1} + Mp_{t-1} + z_{t-1} - \sum_{i \in I}\sum_{j \in J_i} ( e_{i}x_{i(t-1)j} + f_i y_{i(t-1)j})
$$
$$
         0 \le s_t \le  NB
$$
         
$$
    s_0 = 0
$$
3. Bound on $z_t$:
$$
0 \le z_t
$$

### Final model
$$
\left\{
  \begin{array}{rcr}
        \min Nc_b + Mc_p + \sum_{t \in T} z_t \\
        \sum_{i \in I} \sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj}) - Mp_t - s_t \le & z_t & \forall t \in T \\
        s_t = & s_{t-1} + Mp_{t-1} + z_{t-1} - \sum_{i \in I}\sum_{j \in J_i} ( e_{i}x_{i(t-1)j} + f_i y_{i(t-1)j}) & \forall t \in T\\
        0 \le s_t \le & NB & \forall t \in T \\
        s_0 = & 0 \\
        z_t \ge & 0 & \forall t \in T \\
        \sum_{t \in T} x_{itj} \ge & d_i & \forall i \in I, \, \forall j \in J \\
        \sum_{i \in I} \sum_{j \in J_i} e_{i}x_{itj}  \le & m_t & \forall t \in T \\
        x_{itj} = & 0 & \text{for some $t \in T$ and for some $i \in I$, } \forall j \in J_i \\
        \sum_{i \in M'} \sum_{j \in J_i} x_{itj} \le & 1  & \forall t \in T, \, \forall M' \in SHARED\\
        y_{itj} \le x_{itj} \le & y_{itj}+x_{i(t-1)j} & \forall t \in T-\{0\}, \, \forall i \in I, \, \forall j \in J_i \\
        y_{ltj} \cdot d_k  \le & \sum_{t' < t, \, t' \in T} (x_{kt'j}) & \text{ for every dependency }(k,l) \in DEPEN \text{ and $\forall j \in J_l$, $\forall t \in T$} \\
        c_i \cdot \sum_{j \in J_i} y_{itj} \le & \sum_{t-c_i \le t' < t, \, t' \in T}\sum_{j \in J_i} (1-x_{it'j}) & \forall t \in T, \, \forall i \in I \\
        \sum_{t' \le T_{j,i}, \, t' \in T} x_{it'j} \ge & d_i & \forall i \in I, \, \forall j \in J_i\\
        \sum_{j \in J_i} x_{itj} \le & 1 & \forall t \in T, \, \forall i \in I\\
        \sum_{j \in J_i} y_{itj} \le & 1 & \forall t \in T, \, \forall i \in I \\
        x_{ijt} \in & \{0,1\} & \forall t \in T, \, \forall i \in I, \, \forall j \in J \\
        y_{ijt} \in & \{0,1\} & \forall t \in T, \, \forall i \in I, \, \forall j \in J \\
        s_t \in & \mathbb{R} & \forall t \in T \\
        z_t \in & \mathbb{R} & \forall t \in T \\
        N, \, M \in & \mathbb{N} 
  \end{array}
\right.
$$

## PHASE 4
Now that we fixed N and M, we need to obtain an optimal scheduling that reduces the usage of external energy.

This phase can be divided in 2.

### 4.1
If I need to not use at all any form of external energy. It's the same model of Phase 1 but without the objective function.

$$
\left\{
  \begin{array}{rcr}
        \sum_{i \in I}\sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj}) - Mp_t - s_t \le & 0 & \forall t \in T \\
        s_t = & s_{t-1} + Mp_{t-1} - \sum_{i \in I}\sum_{j \in J_i} ( e_{i}x_{i(t-1)j} + f_i y_{i(t-1)j}) & \forall t \in T \\
        0   \le s_t \le & NB & \forall t \in T \\
        s_0 = & 0 \\
        \sum_{t \in T} x_{itj} \ge & d_i & \forall i \in I, \, \forall j \in J \\
        \sum_{i \in I} \sum_{j \in J_i} e_{i}x_{itj}  \le & m_t & \forall t \in T \\
        x_{itj} = & 0 & \text{for some $t \in T$ and for some $i \in I$, } \forall j \in J_i \\
        \sum_{i \in M'} \sum_{j \in J_i} x_{itj} \le & 1  & \forall t \in T, \, \forall M' \in SHARED\\
        y_{itj} \le x_{itj} \le & y_{itj}+x_{i(t-1)j} & \forall t \in T-\{0\}, \, \forall i \in I, \, \forall j \in J_i \\
        y_{ltj} \cdot d_k  \le & \sum_{t' < t, \, t' \in T} (x_{kt'j}) & \text{ for every dependency }(k,l) \in DEPEN \text{ and $\forall j \in J_l$, $\forall t \in T$} \\
        c_i \cdot \sum_{j \in J_i} y_{itj} \le & \sum_{t-c_i \le t' < t, \, t' \in T}\sum_{j \in J_i} (1-x_{it'j}) & \forall t \in T, \, \forall i \in I \\
        \sum_{t' \le T_{j,i}, \, t' \in T} x_{it'j} \ge & d_i & \forall i \in I, \, \forall j \in J_i\\
        \sum_{j \in J_i} x_{itj} \le & 1 & \forall t \in T, \, \forall i \in I\\
        \sum_{j \in J_i} y_{itj} \le & 1 & \forall t \in T, \, \forall i \in I \\
        x_{ijt} \in & \{0,1\} & \forall t \in T, \, \forall i \in I, \, \forall j \in J \\
        y_{ijt} \in & \{0,1\} & \forall t \in T, \, \forall i \in I, \, \forall j \in J \\
        s_t \in & \mathbb{R} & \forall t \in T
  \end{array}
\right.
$$

### 4.3
Minimize the external energy used. It's the same as model 3 but with a modified objective function that doesn't aim to minimize N and M.

$$
\left\{
  \begin{array}{rcr}
        \sum_{i \in I} \sum_{j \in J_i} (e_{i}x_{itj} +f_i y_{itj}) - Mp_t - s_t \le & z_t & \forall t \in T \\
        s_t = & s_{t-1} + Mp_{t-1} + z_{t-1} - \sum_{i \in I}\sum_{j \in J_i} ( e_{i}x_{i(t-1)j} + f_i y_{i(t-1)j}) & \forall t \in T\\
        0 \le s_t \le & NB & \forall t \in T \\
        s_0 = & 0 \\
        z_t \ge & 0 & \forall t \in T \\
        \sum_{t \in T} x_{itj} \ge & d_i & \forall i \in I, \, \forall j \in J \\
        \sum_{i \in I} \sum_{j \in J_i} e_{i}x_{itj}  \le & m_t & \forall t \in T \\
        x_{itj} = & 0 & \text{for some $t \in T$ and for some $i \in I$, } \forall j \in J_i \\
        \sum_{i \in M'} \sum_{j \in J_i} x_{itj} \le & 1  & \forall t \in T, \, \forall M' \in SHARED\\
        y_{itj} \le x_{itj} \le & y_{itj}+x_{i(t-1)j} & \forall t \in T-\{0\}, \, \forall i \in I, \, \forall j \in J_i \\
        y_{ltj} \cdot d_k  \le & \sum_{t' < t, \, t' \in T} (x_{kt'j}) & \text{ for every dependency }(k,l) \in DEPEN \text{ and $\forall j \in J_l$, $\forall t \in T$} \\
        c_i \cdot \sum_{j \in J_i} y_{itj} \le & \sum_{t-c_i \le t' < t, \, t' \in T}\sum_{j \in J_i} (1-x_{it'j}) & \forall t \in T, \, \forall i \in I \\
        \sum_{t' \le T_{j,i}, \, t' \in T} x_{it'j} \ge & d_i & \forall i \in I, \, \forall j \in J_i\\
        \sum_{j \in J_i} x_{itj} \le & 1 & \forall t \in T, \, \forall i \in I\\
        \sum_{j \in J_i} y_{itj} \le & 1 & \forall t \in T, \, \forall i \in I \\
        x_{ijt} \in & \{0,1\} & \forall t \in T, \, \forall i \in I, \, \forall j \in J \\
        y_{ijt} \in & \{0,1\} & \forall t \in T, \, \forall i \in I, \, \forall j \in J \\
        s_t \in & \mathbb{R} & \forall t \in T \\
        z_t \in & \mathbb{R} & \forall t \in T
  \end{array}
\right.
$$


# Algorithms to solve the problem
This file is also available in the folder `scheduling_problem` with the corresponding solution algorithms.